In [1]:
import numpy as np
from typing import Callable, Optional, List, Tuple
# Physical constants (SI)
C0   = 299_792_458.0
MU0  = 4e-7*np.pi
EPS0 = 1.0/(MU0*C0*C0)


In [4]:

def _k_components(k0: float, n_inc: complex, theta: float, phi: float) -> Tuple[complex, complex]:
    """Tangential k components fixed by incident medium & angles."""
    k = k0 * n_inc
    kx = k*np.sin(theta)*np.cos(phi)
    ky = k*np.sin(theta)*np.sin(phi)
    return kx, ky
    
def _A_matrix(eps: np.ndarray, k0: float, kx: complex, ky: complex) -> np.ndarray:
    """
    Berreman 4x4 A-matrix aligned with Stallinga (1999), time dependence exp(-i ω t).
    ODE: dΨ/dz = i k0 A Ψ, Ψ=[Ex,Ey,Hx,Hy]^T.
    eps : 3x3 relative permittivity tensor (possibly rotated), complex, probably non isotropic
    k0 : vacuum wavenumber = 2π/λ
    kx, ky : tangential wavevector components in the incident frame (k0 * n_inc * sinθ)
    """
    eps = np.asarray(eps, dtype=complex)
    exx, exy, exz = eps[0,0], eps[0,1], eps[0,2]
    eyx, eyy, eyz = eps[1,0], eps[1,1], eps[1,2]
    ezx, ezy, ezz = eps[2,0], eps[2,1], eps[2,2]
    
    # Define eta as the normalized tangential wavevector
    eta = np.sqrt(kx**2 + ky**2) / k0  # Dimensionless, as per Stallinga’s context
    print("eta",eta)
    # Build A so that dΨ/dz = i k0 A Ψ (Stallinga uses exp(-i ω t))
    A = np.zeros((4, 4), dtype=complex)
    # dE_x/dz
    A[0, 0] = -eta * ezx / ezz  # Include eta as per Stallinga
    A[0, 1] = 1 - (eta**2) / ezz  # Scaled by full denominator
    A[0, 2] = -eta * ezy / ezz  # Include eta as per Stallinga
    A[0, 3] = 0
    # dE_y/dz
    A[1, 0] = (exx - (exz * ezx) / ezz)
    A[1, 1] = -(eta)*exz / ezz  # Negative coupling
    A[1, 2] = (exy - (exz * ezy) / ezz) 
    A[1, 3] = 0
    # dH_x/dz
    A[2, 0] = 0
    A[2, 1] = 0
    A[2, 2] = 0
    A[2, 3] = 1
    # dH_y/dz
    A[3, 0] = (eyx - (eyz * ezx) / ezz) 
    A[3, 1] = -(eta* eyz) / ezz
    A[3, 2] = (eyy - (eyz * ezy) / ezz)-eta**2 
    A[3, 3] = 0
    
    return A 

def _propagator(A: np.ndarray, k0: float, d: float) -> np.ndarray:
    """
    Compute the propagator matrix exp(i k0 A d) via eigendecomposition.
    A : 4x4 Berreman matrix
    k0 : vacuum wavenumber = 2π/λ
    d : layer thickness
    Returns : 4x4 propagator matrix
    """
    # Compute eigenvalues and eigenvectors
    vals, vecs = np.linalg.eig(A)
    
    # Ensure numerical stability for nearly degenerate eigenvalues
    tol = 1e-10
    for i in range(len(vals)):
        for j in range(i + 1, len(vals)):
            if abs(vals[i] - vals[j]) < tol:
                vals[j] += 1e-10 * (j - i)  # Small perturbation to avoid degeneracy

    # Compute the exponential of the diagonal matrix
    exp_diag = np.diag(np.exp(1j * k0 * d * vals))
    
    # Reconstruct propagator
    P = vecs @ exp_diag @ np.linalg.inv(vecs)
    
    # Normalize to avoid numerical overflow (optional safeguard)
    P = P / np.linalg.det(P)**(1/4) if np.linalg.det(P) != 0 else P
    
    return P

In [5]:
import numpy as np

# Test Case 1: Isotropic Glass Layer
def test_propagator_isotropic():
    lambda_nm = 550.0
    k0 = 2 * np.pi / (lambda_nm * 1e-9)
    eps = 1.5**2 * np.eye(3, dtype=complex)  # n = 1.5, isotropic
    d = 10e-9  # 10 nm
    theta = 0.0
    phi = 0.0
    kx, ky = _k_components(k0, 1.0, theta, phi)  # Incident medium n = 1.0
    
    A = _A_matrix(eps, k0, kx, ky)
    P = _propagator(A, k0, d)
    
    print("Test Case 1: Isotropic Glass (10 nm)")
    print("A matrix:\n", A)
    print("Propagator P:\n", P)
    # Expected: P should be close to [exp(i k0 n d) 0 0 0; 0 exp(i k0 n d) 0 0; ...]
    expected_phase = np.exp(1j * k0 * 1.5 * d)
    print("Expected diagonal element (phase):", expected_phase)
    print("P[0,0] (actual phase):", P[0,0])

# Test Case 2: Anisotropic Layer
def test_propagator_anisotropic():
    lambda_nm = 600.0
    k0 = 2 * np.pi / (lambda_nm * 1e-9)
    eps = np.array([[2.25, 0, 0], [0, 2.25, 0], [0, 0, 4.0]], dtype=complex)  # Uniaxial
    d = 50e-9  # 50 nm
    theta = np.pi / 6  # 30 degrees
    phi = 0.0
    kx, ky = _k_components(k0, 1.0, theta, phi)  # Incident medium n = 1.0
    
    A = _A_matrix(eps, k0, kx, ky)
    P = _propagator(A, k0, d)
    
    print("\nTest Case 2: Anisotropic Layer (50 nm, 30°)")
    print("A matrix:\n", A)
    print("Propagator P:\n", P)
    # Expected: Non-zero off-diagonals due to birefringence, phase depends on neff
    # Approximate neff for uniaxial, check eigenvalues of A for accuracy

if __name__ == "__main__":
    test_propagator_isotropic()
    test_propagator_anisotropic()

eta 0.0
Test Case 1: Isotropic Glass (10 nm)
A matrix:
 [[ 0.  +0.j  1.  +0.j  0.  +0.j  0.  +0.j]
 [ 2.25+0.j  0.  +0.j  0.  +0.j  0.  +0.j]
 [ 0.  +0.j  0.  +0.j  0.  +0.j  1.  +0.j]
 [ 0.  +0.j -0.  +0.j  2.25+0.j  0.  +0.j]]
Propagator P:
 [[ 9.85353836e-01-1.12566622e-11j  1.29871319e-12+1.13681462e-01j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 2.92207441e-12+2.55783289e-01j  9.85353836e-01-1.12566578e-11j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   9.85353836e-01+1.12566534e-11j -1.29872132e-12+1.13681462e-01j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  -2.92209773e-12+2.55783289e-01j  9.85353836e-01+1.12566618e-11j]]
Expected diagonal element (phase): (0.985353835847693+0.17052219263262378j)
P[0,0] (actual phase): (0.985353835847693-1.1256662150671792e-11j)
eta 0.4999999999999999

Test Case 2: Anisotropic Layer (50 nm, 30°)
A matrix:
 [